# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Copy Files

In [ ]:
!cp 'drive/My Drive/ITToday/'*.csv .

# Sample Submission

In [ ]:
import pandas as pd
example = pd.read_csv('sampleSubmission.csv')
example.head(10)

,id,Lng,Lat
0,516,106.733964,-6.557158
1,517,106.733964,-6.557158
2,518,106.733964,-6.557158
3,519,106.733964,-6.557158
4,520,106.733964,-6.557158
5,521,106.733964,-6.557158
6,522,106.733964,-6.557158
7,523,106.733964,-6.557158
8,524,106.733964,-6.557158
9,525,106.733964,-6.557158


# Load Data

In [ ]:
from pandas_profiling import ProfileReport as Report

QuadData = pd.read_csv('QuadData_train.csv')
to_pred  = pd.read_csv('QuadData_test.csv')

print('train data shape:', QuadData.shape)
print('test data shape :', to_pred.shape)

train data shape: (6440, 47)
test data shape : (2351, 47)


In [ ]:
Report(QuadData)

Number of variables,47
Number of observations,6440
Total Missing (%),0.0%
Total size in memory,2.3 MiB
Average record size in memory,376.0 B
Numeric,44
Categorical,1
Boolean,2
Date,0
Text (Unique),0
Rejected,0


In [ ]:
dev_dict = {'F330': 0, 'F450': 1}
QuadData = QuadData.replace({'DeviceID': dev_dict})
to_pred  = to_pred.replace({'DeviceID': dev_dict})
QuadData.head(2)

,id,ExpID,PatternID,DeviceID,LineNo_ATT,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,ErrRP,ErrYaw,ThI,ABst,ThH,DAlt,TAlt,DCRt,CRt,VDop,HAcc,VAcc,SAcc,Delta,Status,GMS,NSats,Alt.1,Spd,GCrs,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,AccZ_2,MagX,MagY,MagZ,MagZ_2,C1,C2,C3,Lat,Lng
0,0,6,2,1,14206,-2.52,-0.74,-2.30,-1.55,359.96,2.12,0.10,0.03,0.555876,0.000247,0.497376,-0.296357,1.502355,-56,-55,1.12,0.58,0.96,0.21,200,4,554817200,14,215.19,0.391,187.202,0.078031,-0.032413,-0.226329,0.226690,-0.065972,-10.823730,-10.886220,382,-12,-200,-275,1536,1578,1719,-6.557158,106.733964
1,1,6,2,1,14216,-2.64,0.28,-2.95,-1.02,359.96,359.69,0.12,0.02,0.540934,0.000086,0.497376,-0.396357,1.403758,-56,-56,1.12,0.58,0.95,0.25,200,4,554817400,14,215.10,0.242,187.202,0.048311,-0.123246,-0.176451,1.294079,-0.242524,-9.683836,-9.850414,385,4,-193,-277,1581,1553,1697,-6.557158,106.733964


# Extract the Data

In [ ]:
def extractData(data, to_drop):
    y = data[to_drop] #what we want to predict
    x = data.drop(to_drop, axis = 1)
    return x, y

In [ ]:
ys = ['Lng', 'Lat']
x, y = extractData(QuadData, ys)
to_pred = to_pred.drop(ys, axis = 1)
print(x.shape, y.shape, to_pred.shape)

(6440, 45) (6440, 2) (2351, 45)


# Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x = scaler.fit_transform(x)
to_pred = scaler.transform(to_pred)

pd.DataFrame(x).head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0.000000,0.454545,1.0,1.0,0.487960,0.229194,0.227626,0.409800,0.316136,0.999917,0.005862,0.5,0.040541,0.430157,0.788327,0.640642,0.131457,0.102921,0.32,0.412602,0.407407,0.533333,0.424242,0.226415,0.344262,1.0,0.946879,0.181818,0.519016,0.072911,0.520028,0.377304,0.514386,0.400447,0.467649,0.873730,0.509687,0.602476,0.954545,0.549531,0.378182,0.223796,0.627164,0.689516,0.870839
1,0.000119,0.454545,1.0,1.0,0.488314,0.227213,0.242566,0.400903,0.323257,0.999917,0.999222,0.6,0.027027,0.423738,0.787803,0.640642,0.126588,0.097902,0.32,0.410569,0.407407,0.533333,0.409091,0.301887,0.344262,1.0,0.946890,0.181818,0.515660,0.045055,0.520028,0.369660,0.473076,0.417251,0.538817,0.861961,0.551377,0.649525,0.957576,0.566215,0.390909,0.218130,0.687084,0.655914,0.841545


# Define Custom Scorer

In [ ]:
!pip install haversine

In [ ]:
from sklearn.metrics.scorer import make_scorer
from haversine import haversine

def haversine_score(y_true, y_pred):
  assert len(y_true) == len(y_pred)
  total, n = 0, len(y_true)
  for i in range(n):
    total += haversine(y_true[i], y_pred[i])
  return total/n

haversine_scorer = make_scorer(haversine_score, greater_is_better = False)

# The Regressors before Feature Engineering

In [ ]:
import numpy as np
from math import sqrt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, Lasso, LassoLars, ElasticNet, OrthogonalMatchingPursuit
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor

from mlxtend.regressor import StackingRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
knn = KNeighborsRegressor()

linreg = LinearRegression()

linearSVR = LinearSVR(random_state = 123)
kernelSVR = SVR()

extratree = ExtraTreeRegressor(random_state = 123)
decisiontree = DecisionTreeRegressor(random_state = 123)

randomforest = RandomForestRegressor(random_state = 123, n_estimators = 100)
randomforest2 = RandomForestRegressor(random_state = 123, n_estimators = 100, max_features = "log2")

ridge = Ridge(random_state = 123)
bayesridge = BayesianRidge()

lasso = Lasso()
lars = LassoLars()
net = ElasticNet()

omp = OrthogonalMatchingPursuit()

bagging = BaggingRegressor(random_state= 123, n_estimators = 100)

boosting = GradientBoostingRegressor(random_state = 123, n_estimators = 100)
adaboost = AdaBoostRegressor(random_state= 123, n_estimators= 100)

In [ ]:
print('7-fold cross validation:\n')

for clf, label in zip([knn, linreg, extratree, decisiontree, randomforest,
                       randomforest2, lasso, lars, omp, bagging],
                      ['K-Neighbors Regressor',
                       'Linear Regression',
                       'Extra Tree Regressor',
                       'Decision Tree Regressor',
                       'Random Forest Regressor 1',
                       'Random Forest Regressor 2',
                       'Lasso',
                       'Lasso Lars',
                       'Orthogonal Matching Pursuit',
                       'Bagging Regressor']):

    try:
        scores = cross_val_score(clf, x, np.array(y), cv = 7, scoring = haversine_scorer)
        print("Mean Haversine Distance: %0.4f (+/- %0.4f) [%s]"
              % (abs(scores.mean()), abs(scores.std()), label))
    except Exception as e:
        print(type(e).__name__ + " [%s]"%(label))
        if str(e).strip() != '':
            print('\t' + str(e))
        continue

7-fold cross validation:

Mean Haversine Distance: 0.0087 (+/- 0.0053) [K-Neighbors Regressor]
Mean Haversine Distance: 0.0180 (+/- 0.0093) [Linear Regression]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Extra Tree Regressor]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Decision Tree Regressor]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Random Forest Regressor 1]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Random Forest Regressor 2]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Lasso]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Lasso Lars]
Mean Haversine Distance: 0.0133 (+/- 0.0013) [Orthogonal Matching Pursuit]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Bagging Regressor]


# Dimension Reduction

In [ ]:
x = pd.DataFrame(x)
x.columns = pd.read_csv('QuadData_train.csv').columns[:-2]

x.head(2)

,id,ExpID,PatternID,DeviceID,LineNo_ATT,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,ErrRP,ErrYaw,ThI,ABst,ThH,DAlt,TAlt,DCRt,CRt,VDop,HAcc,VAcc,SAcc,Delta,Status,GMS,NSats,Alt.1,Spd,GCrs,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,AccZ_2,MagX,MagY,MagZ,MagZ_2,C1,C2,C3
0,0.000000,0.454545,1.0,1.0,0.487960,0.229194,0.227626,0.409800,0.316136,0.999917,0.005862,0.5,0.040541,0.430157,0.788327,0.640642,0.131457,0.102921,0.32,0.412602,0.407407,0.533333,0.424242,0.226415,0.344262,1.0,0.946879,0.181818,0.519016,0.072911,0.520028,0.377304,0.514386,0.400447,0.467649,0.873730,0.509687,0.602476,0.954545,0.549531,0.378182,0.223796,0.627164,0.689516,0.870839
1,0.000119,0.454545,1.0,1.0,0.488314,0.227213,0.242566,0.400903,0.323257,0.999917,0.999222,0.6,0.027027,0.423738,0.787803,0.640642,0.126588,0.097902,0.32,0.410569,0.407407,0.533333,0.409091,0.301887,0.344262,1.0,0.946890,0.181818,0.515660,0.045055,0.520028,0.369660,0.473076,0.417251,0.538817,0.861961,0.551377,0.649525,0.957576,0.566215,0.390909,0.218130,0.687084,0.655914,0.841545


In [ ]:
x = x.drop('id', axis = 1)
x.head(2)

,ExpID,PatternID,DeviceID,LineNo_ATT,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,ErrRP,ErrYaw,ThI,ABst,ThH,DAlt,TAlt,DCRt,CRt,VDop,HAcc,VAcc,SAcc,Delta,Status,GMS,NSats,Alt.1,Spd,GCrs,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,AccZ_2,MagX,MagY,MagZ,MagZ_2,C1,C2,C3
0,0.454545,1.0,1.0,0.487960,0.229194,0.227626,0.409800,0.316136,0.999917,0.005862,0.5,0.040541,0.430157,0.788327,0.640642,0.131457,0.102921,0.32,0.412602,0.407407,0.533333,0.424242,0.226415,0.344262,1.0,0.946879,0.181818,0.519016,0.072911,0.520028,0.377304,0.514386,0.400447,0.467649,0.873730,0.509687,0.602476,0.954545,0.549531,0.378182,0.223796,0.627164,0.689516,0.870839
1,0.454545,1.0,1.0,0.488314,0.227213,0.242566,0.400903,0.323257,0.999917,0.999222,0.6,0.027027,0.423738,0.787803,0.640642,0.126588,0.097902,0.32,0.410569,0.407407,0.533333,0.409091,0.301887,0.344262,1.0,0.946890,0.181818,0.515660,0.045055,0.520028,0.369660,0.473076,0.417251,0.538817,0.861961,0.551377,0.649525,0.957576,0.566215,0.390909,0.218130,0.687084,0.655914,0.841545


In [ ]:
to_pred = pd.DataFrame(to_pred)
to_pred.columns = pd.read_csv('QuadData_train.csv').columns[:-2]
to_pred = to_pred.drop('id', axis = 1)
to_pred.head(2)

,ExpID,PatternID,DeviceID,LineNo_ATT,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,ErrRP,ErrYaw,ThI,ABst,ThH,DAlt,TAlt,DCRt,CRt,VDop,HAcc,VAcc,SAcc,Delta,Status,GMS,NSats,Alt.1,Spd,GCrs,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,AccZ_2,MagX,MagY,MagZ,MagZ_2,C1,C2,C3
0,0.454545,1.0,1.0,0.670415,0.253137,0.214589,0.306050,0.231896,0.020696,0.012418,0.3,0.162162,0.626115,0.819777,0.844599,0.528084,0.439816,0.931111,0.591463,0.407407,0.6,0.545455,0.094340,0.344262,1.0,0.956279,0.181818,0.584265,0.420639,0.031854,0.185190,0.583468,0.495136,0.442400,0.837240,0.503601,0.654420,0.932323,0.529718,0.310909,0.076487,0.652463,0.830645,0.868176
1,0.454545,1.0,1.0,0.670768,0.251321,0.147649,0.326444,0.239285,0.016418,0.014529,0.3,0.175676,0.582689,0.829492,0.844599,0.528084,0.449099,0.913333,0.648374,0.407407,0.6,0.545455,0.132075,0.344262,1.0,0.956290,0.181818,0.593214,0.459525,0.026536,0.377181,0.548597,0.524757,0.384225,0.849504,0.523404,0.608385,0.926263,0.562044,0.325455,0.110482,0.591212,0.713710,0.793609


In [ ]:
y = pd.DataFrame(y)
y.columns = ['Lng', 'Lat']

y.head(2)

,Lng,Lat
0,106.733964,-6.557158
1,106.733964,-6.557158


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 9)
X = pca.fit_transform(x)
X = pd.DataFrame(X)

X.head(2)

,0,1,2,3,4,5,6,7,8
0,0.334505,-0.053698,-0.333276,0.807016,0.132924,-0.552934,0.466689,0.012745,-0.150100
1,0.374938,0.515830,-0.493993,0.803071,0.031128,-0.588360,0.576163,-0.239119,-0.100651


In [ ]:
to_pred = pca.transform(to_pred)
to_pred.head(2)

AttributeError: ignored

# Regressors after Feature Dimension Reduction

In [ ]:
print('7-fold cross validation:\n')

for clf, label in zip([knn, linreg, extratree, decisiontree, randomforest,
                       randomforest2, lasso, lars, omp, bagging],
                      ['K-Neighbors Regressor',
                       'Linear Regression',
                       'Extra Tree Regressor',
                       'Decision Tree Regressor',
                       'Random Forest Regressor 1',
                       'Random Forest Regressor 2',
                       'Lasso',
                       'Lasso Lars',
                       'Orthogonal Matching Pursuit',
                       'Bagging Regressor']):

    try:
        scores = cross_val_score(clf, X, np.array(y), cv = 7, scoring = haversine_scorer)
        print("Mean Haversine Distance: %0.4f (+/- %0.4f) [%s]"
              % (abs(scores.mean()), abs(scores.std()), label))
    except Exception as e:
        print(type(e).__name__ + " [%s]"%(label))
        if str(e).strip() != '':
            print('\t' + str(e))
        continue

7-fold cross validation:

Mean Haversine Distance: 0.0097 (+/- 0.0047) [K-Neighbors Regressor]
Mean Haversine Distance: 0.0149 (+/- 0.0009) [Linear Regression]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Extra Tree Regressor]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Decision Tree Regressor]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Random Forest Regressor 1]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Random Forest Regressor 2]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Lasso]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Lasso Lars]
Mean Haversine Distance: 0.0170 (+/- 0.0020) [Orthogonal Matching Pursuit]
Mean Haversine Distance: 0.0181 (+/- 0.0033) [Bagging Regressor]


# Grid Search Optimization

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
KNeighborsRegressor()

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [ ]:
knn_param = {
    'n_neighbors' : list(range(1, 21)),
    'weights' : ['uniform', 'distance']
}

In [ ]:
gridknn = GridSearchCV(knn, knn_param, cv = 7, scoring = haversine_scorer)
gridknn.fit(x, np.array(y));
print("(best score: %s, best parameters: %s)"%(gridknn.best_score_, gridknn.best_params_))

(best score: -0.008660512125199787, best parameters: {'n_neighbors': 5, 'weights': 'distance'})


In [ ]:
gridknn = GridSearchCV(knn, knn_param, cv = 7, scoring = haversine_scorer)
gridknn.fit(X, np.array(y));
print("(best score: %s, best parameters: %s)"%(gridknn.best_score_, gridknn.best_params_))

(best score: -0.009677476484468579, best parameters: {'n_neighbors': 7, 'weights': 'distance'})


# Prediction

In [ ]:
data = X
target = y

model = KNeighborsRegressor(n_neighbors = 7, weights = 'distance')

model.fit(data, target)
Preds = model.predict(to_pred)

Out = pd.DataFrame({'id': pd.read_csv('QuadData_test.csv')['id']})
Out['Lng'] = [Preds[el][0] for el in range(2351)]
Out['Lat'] = [Preds[el][1] for el in range(2351)]
Out.head()

,id,Lng,Lat
0,516,106.734237,-6.557377
1,517,106.734238,-6.557373
2,518,106.734238,-6.557372
3,519,106.734237,-6.557375
4,520,106.734237,-6.557261


In [ ]:
Out.to_csv('Submission 05_Optimized KNN_w_PCA_w_MinMaxScaler.csv', index = False)
!cp 'Submission 05_Optimized KNN_w_PCA_w_MinMaxScaler.csv' 'drive/My Drive/ITToday/Submissions'

In [ ]:
Submission = pd.read_csv('Submission 05_Optimized KNN_w_PCA_w_MinMaxScaler.csv')
Submission.head()

,id,Lng,Lat
0,516,106.734237,-6.557377
1,517,106.734238,-6.557373
2,518,106.734238,-6.557372
3,519,106.734237,-6.557375
4,520,106.734237,-6.557261
